In [ ]:
import pandas as pd
import numpy
import codecs
import csv
import json
import pprint
import re

filename = "math sample for API testing.csv"

fields ={'Internal_ID': 'Internal_ID',
         'Strand': 'Strand',
         'Sub-Strand': 'Substrand',
         'Content Focus': 'Content_Focus',
         'Component': 'Component',
         'Number': 'Number',
         'Digits': 'Digits',
         'Shape': 'Shape',
         'Units': 'Units',
         'Steps': 'Steps',
         'Context': 'Context',
         'SIG Stem': 'SIG_Stem',
         'SIG Options': 'SIG_Options',
         'Additional Modifiers': 'Additional_Modifiers'}

In [ ]:
def process_file(filename, fields, tagging_df):
    process_fields = fields.keys()
    data = []
    holding = []
#    data_keys = ['Internal ID', 'Strand','Substrand', 'Content_focus', 'Component', 'Number', 'Digits', 'Shape', 'Units', 'Steps', 'Context', 'SIG Stem', 'SIG Options', 'Additional Modifiers']
    with open(filename, "r") as f:
        reader = csv.DictReader(f)
        for i in range(1):
            l = reader.next()

        for line in reader:
            holding_test = 'FALSE'
            row = {}
            strand_dict = {}
            strand_list = []
            InternalID = int(line['Internal ID'])
            Strand = line['Strand'].split("; ")
            for strand in Strand:
                strand_sub_dict = {}
                content_focus_df = tagging_df['Content_Focus']
                StrandGUID = get_GUID_lite(strand, content_focus_df)
                strand_sub_dict = {'Strand': strand, 'StrandGUID' : StrandGUID}
                strand_list.append(strand_sub_dict)
            #strand_dict = {"Strand": strand_list} 
                   
            #row['Substrand'] = line['Sub-Strand'].split("; ")
            #Substrand = line['Sub-Strand'].split("; ")
            #for substrand in Substrand:
            #    SubstrandGUID = get_SubstrandGUID(strand_list, substrand, content_focus_df)
            #    substrand_sub_dict = {'Sub-strand': substrand, 'Sub-strandGUID': SubstrandGUID}
                #Need to place substrand underneath correct dictionary element in strand list
                 
                
            Differentiator_Keys = ['Number', 'Shape', 'Units', 'Steps']            
            Differentiator_List = []
            for Differentiator in Differentiator_Keys:
                Differentiator_values = line[Differentiator].split("; ")
                Differentiator_sub_list = []
                Differentiator_df = tagging_df[Differentiator]
                DifferentiatorGUID_Name = Differentiator + "GUID"
                for value in Differentiator_values:
                    Differentiator_sub_dict = {}
                    DifferentiatorGUID = get_GUID_lite(value, Differentiator_df)
                    Differentiator_sub_dict = {Differentiator: value, DifferentiatorGUID_Name : DifferentiatorGUID}
                    Differentiator_sub_list.append(Differentiator_sub_dict)
                Differentiator_List.append(Differentiator_sub_list)
            #Process Differentiators
            
            
#            if SubstrandGUID contains "Error":
#            holding_test == 'TRUE'        
            
            row = {"Internal_ID": InternalID, "Organizers" : strand_list, "Differentiators": Differentiator_List}
            
    #At end, test to determine whether any element in dictionary starts with "Error". If so, change holding_value
                    
#            for key,value in fields.items():
#                row[value] = (line[key]).split("; ")
#                #If strand, substrand, Content Focus or Numbers field in raw data has ";", then add to holding list
#                if (value == "Strand" or value == "Substrand" or value == "Content_Focus" or value == "Number") and (re.search(';', line[key])):
#                    holding_test = 'TRUE'
#            if holding_test == 'TRUE':
#                holding.append(row) 
#            else:  
            data.append(row)
    #Process holding list
    #Do the row have multiple values in strand both and substrand? If yes, then consult table_of_values to figure out what should get 
    #paired together
    #pprint.pprint(holding)
    return data

In [ ]:
data = process_file(filename, fields, tagging_dict)
pprint.pprint(data)

In [ ]:
pprint.pprint(data)

In [ ]:
def get_SubstrandGUID(strand_list, substrand, df):
    
    substrand = unicode(substrand, "latin2").lower()
    if len(substrand) == 1 and len(strand_list) > 1:
        SubStrandGUID = "Error: more strands than substrands"
        break
    elif len(substrand) == 1 and len(strand_list) == 1:
        strand = unicode(strand_list[0], "latin2").lower()
        matches = len(df[(df['Strand']==strand) & (df['Title']==substrand)])
        if matches == 1:
            GUID = df.ix[(df['Strand']==strand) & (df['Title']==substrand), 'Unique ID'].item()
        else: 
            GUID = 'Error: Substrand lives in multiple strands'
    return GUID



            
                    SubstrandGUID = get_SubstrandGUID(strand, substrand, content_focus_df)


In [ ]:
#Make every internal ID of int data type "Internal ID"

In [ ]:
#load GUID file

In [ ]:
#problem to solve for looking up GUID: when strand/substrand gives context to component, 
#if semi-colon separated, then will have to allow for non-matches

In [ ]:
def Load_GUIDs(filename):
    data = []
    fields = {}
    with open(filename, "r") as f:
        reader = csv.DictReader(f)
        for line in reader:
            data.append(line)
    #strip capitalization for matching purposes
    data = pd.DataFrame(data)
    df_lowercase(data, 'Title')
    if len(data.columns) == 6:
        df_lowercase(data, 'Strand')
        df_lowercase(data, 'Sub-Strand')
    if len(data.columns) == 7:
        df_lowercase(data, 'Strand')
    return data

In [ ]:
def df_lowercase(dataframe, column):
    lowercase = lambda x: unicode(x, "latin2").lower()
    dataframe[column] = dataframe[column].apply(lowercase)
    return dataframe

In [ ]:
steps_df = Load_GUIDs("Math_Steps.csv")
shapes_df = Load_GUIDs("Math_Shapes.csv")
number_df = Load_GUIDs("Math_Number.csv")
units_df = Load_GUIDs("Math_Units_Modified.csv")
content_focus_df = Load_GUIDs("Math_Content_Foc.csv")
component_df = Load_GUIDs("Math_Components.csv")
tagging_dict = {'Steps': steps_df, 'Shape': shapes_df, 'Number': number_df, 'Units': units_df, 'Content_Focus': content_focus_df, 'Component': component_df}

In [ ]:

def get_GUID_lite(tag, df): 
    #get GUID for steps, number, units, shape
    tag = unicode(tag, "latin2").lower()
    matches = len(df.ix[df['Title'] == tag])
    if matches == 1:
        GUID = df.ix[df['Title'] == tag, 'Unique ID'].item()
    elif matches ==0:
        if tag == r'n/a':
            GUID = r'N/A'
        else: GUID = "Error: no matches found"
    else: GUID = "Error: resolve multiple values"
    return GUID

In [ ]:
number_test = "Square roots"
print get_GUID_lite(number_test, number_df)

In [ ]:

#create version for content focus and component
#Play around with passing multiple conditions into first .ix argument

tag = 'N/A'
if tag == r'N/A':
    print tag


In [ ]:
'''
Data structure

{Internal ID: value
    {Organizers: [
        {Strand: [ {Strand: Strand_value,
                   Strand_GUID: GUID,
                   Substrand: [{Substrand: Substrand_value,
                                Substrand_GUID: GUID,
                                Content_Focus: [{Content_Focus: Content_Focus_value,                       
                                                Content_Focus_GUID: GUID,
                                                Component: [{Component: Component_value,
                                                              Component_GUID: GUID}]}]}]}]
        {Descriptors: {Number: [N1, N2, ...]}
            {Digits: [D1, D2, ...]}
            {Shape: [Shape1, ...]}
        {Units: [Units1, ...]}
        {Steps: Steps}
        {Context: Context}
        {SIG Stem: [Sig Stem1, ...]}
        {SIG Options: [SIG Options1, ...]}
        {Additional Modifiers: [AM1, ...]}
}

'''